In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import time
import math

from model import Caption_Model
from data_generator import Data_Generator
from inference_util import Inference

import configuration
import inception_base
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--eval_interval_secs", help="Interval between evaluation runs.",
                    type=int, default=300)
parser.add_argument("--min_global_step", help="Minimum global step to run evaluation.",
                    type=int, default=100)
parser.add_argument("--batch_size", help="Number of batches to evaluate at once.",
                    type=int, default=100)
parser.add_argument("--max_batch_process", help="Total number of batches to evaluate.",
                    type=int, default=10)
parser.add_argument("--eval_all_models", help="Whether to evaluate all models in checkpoint_dir",
                    action="store_true")
FLAGS = parser.parse_args(["--eval_all_models"])

In [3]:
# Create the evaluation directory if it doesn't exist.

data_config = configuration.DataConfig().config
data_gen = Data_Generator(data_config["processed_video_dir"],
                      data_config["caption_file"],
                      data_config["unique_frequency_cutoff"],
                      data_config["max_caption_length"])

data_gen.load_vocabulary(data_config["caption_data_dir"])
data_gen.load_dataset(data_config["caption_data_dir"])

FLAGS.checkpoint_dir = data_config["checkpoint_dir"]

eval_dir = data_config["val_log_dir"]

evaluated_models = set([])

In [4]:
model_config = configuration.ModelConfig(data_gen).config
model = Caption_Model(mode="val", **model_config)
model.build()

# Create the Saver to restore model Variables.
saver = tf.train.Saver()

In [5]:
FLAGS.checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

In [6]:
model_path = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

In [7]:
sess = tf.Session()

In [8]:
saver.restore(sess, model_path)

INFO:tensorflow:Restoring parameters from ./models/train/model-5


In [9]:
data_gen.init_batch(FLAGS.batch_size,"val")
total_loss = 0

global_step = sess.run(model.global_step)

start_time = time.time()
if FLAGS.max_batch_process:
    max_iter = FLAGS.max_batch_process
else:
    max_iter = data_gen.iter_per_epoch["val"]

In [10]:
i=0
dataset = data_gen.get_next_batch("val")
feed_dict={}
feed_dict[model.video_mask] = np.ones([dataset["video"].shape[0],dataset["video"].shape[1]],dtype=np.int32)
feed_dict[model.caption_input] = dataset["indexed_caption"]
feed_dict[model.caption_mask] = dataset["caption_mask"]
feed_dict[model.rnn_input] = dataset["video"]
feed_dict[model.is_training] = False

In [11]:
batch_loss = sess.run(model.batch_loss,feed_dict=feed_dict)

In [ ]:
%debug